Refer here for instructions to [download and run this sample locally](https://developers.arcgis.com/python/sample-notebooks/#Download-and-run-the-sample-notebooks) on your computer

# Using Geoprocessing tools

This sample notebooks shows how geoprocessing tools can be used in an analysis. The analysis below uses a geoprocessing tool to  deduce the path that the debris of a crashed airplane would take if it went down at different places in the ocean. It should be noted that this tool is for the purpose of demonstrating GIS functionality and the results should not be treated as authoritative. Further this tool excludes wind or atmospheric conditions, which are critical for a real-world analysis.

In [1]:
from arcgis.gis import GIS
from IPython.display import display

In [2]:
gis = GIS()
map1 = gis.map()
map1

There have been speculations that the unfortunate disappearance of flight [MH370](https://en.wikipedia.org/wiki/Malaysia_Airlines_Flight_370) is tied to [Diego Garcia](https://en.wikipedia.org/wiki/Diego_Garcia) an atoll in the central Indian Ocean, in addition to other search locations in southern Indian Ocean to west of Australia. The code below plots these locations on the map:

In [3]:
# Use geocoder to find the co-ordinates of Diego Garcia
diegogarcia = gis.tools.geocoders[0].find_best_match("Diego Garcia")
map1.center = diegogarcia

In [4]:
# Annotate the map by plotting Diego Garcia, and two other search locations
map1.draw(gis.tools.geocoders[0].find_best_match("La Reunion"), {"title": "Reunion Island", "content": "Debris found"})
map1.draw(diegogarcia, {"title": "Diego Garcia", "content": "Naval Support Facility Diego Garcia"})
map1.draw([-43.5, 90.5], {"title":"Search Location", "content":"Predicted crash location"})

# Render a feature layer representing the search area
#Source: http://www.amsa.gov.au/media/incidents/images/DIGO_00718_01_14.jpg
map1.add_layer({"type":"FeatureLayer", 
                "url" : "http://services.arcgis.com/WQ9KVmV6xGGMnCiQ/arcgis/rest/services/MH370Search/FeatureServer/1"})

We can search ArcGIS Online for content related to the missing airliner, and bring in layers that we want to use for our analysis and visualization:

In [5]:
mh370items = gis.content.search("MH370", "feature service", max_items=6)
for item in mh370items:
    display(item)

<Item title:"MH370_Flight_Path" type:Feature Service owner:bflanagan_bureau>

<Item title:"Arc" type:Feature Service owner:bflanagan_bureau>

<Item title:"MH370Search" type:Feature Service owner:bflanagan_bureau>

<Item title:"ActualFlightRoute" type:Feature Service owner:bflanagan_bureau>

<Item title:"SatelliteLocation" type:Feature Service owner:bflanagan_bureau>

<Item title:"mh370debris" type:Feature Service owner:bflanagan_bureau>

Let's add a few of these layers to the map for analysis

In [6]:
map1.add_layer(mh370items[0])
map1.add_layer(mh370items[4])
map1.add_layer(mh370items[5])

In [7]:
map1.zoom = 6

# Using geoprocessing tools

Next, we search ArcGIS Online for the Ocean Currents geoprocessing tool that calculates the path of an unpowered vessel drifiting in ocean currents from a user defined starting point. This tool is for demonstration of GIS functionality only and excludes wind or atmospheric conditions, which are critical for a real-world analysis:

In [8]:
gpsvcs = gis.content.search("Ocean Currents", item_type = "geoprocessing service", max_items=1)
for gpsvc in gpsvcs:
    display(gpsvc)

<Item title:"Ocean Currents" type:Geoprocessing Service owner:esri_apac>

We create a **`GeoprocessingTool`** object out of this service using the `arcgis.tools` module

In [9]:
from arcgis.tools import GeoprocessingTool
gptool = GeoprocessingTool(gpsvcs[0])

display(gptool)

URL: http://sampleserver1.arcgisonline.com/ArcGIS/rest/Services/Specialty/ESRI_Currents_World/GPServer
Task: MessageInABottle
Input_Point : GPFeatureRecordSetLayer
Days : GPDouble


GeoprocessingTool([('serviceDescription',
                    'Use this service to calculate the path a bottle drifting in an ocean current would follow given a user defined point and the number of days the bottle travels. This is a sample service hosted by ESRI, powered by ArcGIS Server. ESRI has provided this example so that you may practice using ArcGIS APIs for JavaScript, Flex, and Silverlight. ESRI reserves the right to change or remove this service at any time and without notice. '),
                   ('tasks', ['MessageInABottle']),
                   ('executionType', 'esriExecutionTypeSynchronous'),
                   ('resultMapServerName', '')])

The ArcGIS Python API dynamically adds methods (in this case the **`message_in_a_bottle()`** method) based on the tool that is constructed. Such methods are added with documentation strings from the geoprocessing service and can be queried through intellisence.

From the output of the previous cell, we can see, the geoprocessing tool requires 2 input parameters: a `GPFeatureRecordSetLayer` which contains the input location as a point feature and a `GPDouble` parameter containing the number of days to be used for prediction. A `GPFeatureRecordSetLayer` can be provided as a `FeatureSet` object from `arcgis.lyr` module.

When we click the map, we want to invoke this geoprocessing tool and pass in the clicked location. We can do this by adding an event listener to the map using `map.on_click()`. The event listener then calls the tool with the clicked location.

The analysis results from this tool would contain the paths that the debris would take if the plane went down at the clicked point. Next, we draw the path on the map using `map.draw()` as shown in the code below

In [10]:
from arcgis.tools import FeatureSet, Feature
def do_analysis(m, g):
    #Parameter `g` contains the co-ordinates of the clicked location
    
    #Construct a FeatureSet object from the clicked locaiton
    my_feature_set = FeatureSet([Feature(g)])
    
    #Pass the input location as a FeatureSet to the service for computation
    ret = gptool.message_in_a_bottle(my_feature_set, 150)
    
    #Render the resulting FeatureSet on the map using `draw()` method
    map1.draw(ret)
    
#Call the method that performs analysis whenever user clicks on the map.
map1.on_click(do_analysis)

Click around on the map, near Diego Garcia as well as at the search locations to find the paths the debris could have taken and find if it could have reached La Reunion.

**Note**: The tool expects an input location that is within the worlds oceans and will not work if clicked within land masses. To know more about this tool, visit the [help documentation](http://sampleserver1b.arcgisonline.com/arcgisoutput/Specialty_ESRI_Currents_World/MessageInABottle.htm).